In [1]:
%%capture

from sz_utils import data_handler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# check if gpu is available
import tensorflow as tf
tf.config.list_physical_devices('GPU')

# collect the data
preictal, interictal = data_handler.make_patient_windows("chb01")

# make the labels
X = np.concatenate((preictal, interictal), axis=0)
y = np.concatenate((np.ones((preictal.shape[0], 1)), np.zeros((interictal.shape[0], 1))), axis=0)

# Shuffle the data
shuffle_indices = np.random.permutation(np.arange(X.shape[0]))
X = X[shuffle_indices]
y = y[shuffle_indices]

# Split the data into train and test
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

# Split test data into validation and test
val_size = int(X_test.shape[0] * 0.5)
X_val = X_test[:val_size]
y_val = y_test[:val_size]
X_test = X_test[val_size:]
y_test = y_test[val_size:]

In [2]:
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,LSTM, Conv1D, Activation, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.metrics import AUC
import keras

In [3]:
# shapes
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape, "X_val shape:", X_val.shape, "y_val shape:", y_val.shape, "X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (672, 1280, 22) y_train shape: (672, 1) X_val shape: (84, 1280, 22) y_val shape: (84, 1) X_test shape: (84, 1280, 22) y_test shape: (84, 1)


In [8]:
num_classes = 2 
input_shape_dataset = (X_train.shape[1], X_train.shape[2])
input_shape_dataset

(1280, 22)

In [7]:
num_classes = 2 
input_shape_dataset = (X_train.shape[1], X_train.shape[2])
input_shape_dataset

def create_model_cnn_basic_1_layer(
    input_shape_dataset: tuple = input_shape_dataset,
    num_classes: int = num_classes,
    debug: bool = False,
    filters: int = 256,
    kernel_size: int = 3,
    pool_size: int = 2,
    dropout: float = 0.1,
    dense_size: int = 64,
    loss: str = "binary_crossentropy",
    optimizer: str = "adam",
    metrics: list = ["accuracy"],

) -> tf.keras.Model:

    """This function creates a basic convolutional neural network model with 2 convolutional layers, 2 dense layers and a softmax layer

    :param input_shape_dataset: shape of the input data
    :type input_shape_dataset: tuple
    :param num_classes: number of classes
    :type num_classes: int
    :return: return a model
    :rtype: tf.keras.Model
    """

    if debug:
        print("------------model summary---------------")
        print("input_shape_dataset", input_shape_dataset)
        print("num_classes", num_classes)

    input_shape_dataset: tuple
    
    model = Sequential()

    model.add(Conv1D(filters, kernel_size, input_shape=(input_shape_dataset)))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model

In [86]:
from dataclasses import dataclass
import tensorflow as tf
import mlflow


# @dataclass
class Experimento:
    experiment_name: str
    model_name: str
    model: tf.keras.Model
    dataset: tuple
    hyperparameters: dict
    metrics: dict


    def __init__(self, experiment_name, model_name, model, dataset, hyperparameters, metrics):
        self.experiment_name = experiment_name
        self.model_name = model_name
        self.model = model
        self.dataset = dataset
        self.hyperparameters = hyperparameters
        self.metrics = metrics      

    def fit(self):
        # history = self.model.fit(  self.dataset[0],
        #                             self.dataset[1],
        #                             validation_data=(self.dataset[2], self.dataset[3]),
        #                             epochs=self.hyperparameters["epochs"],
        #                         )
        # return history

        # with mlflow.start_run(nested=True):
        mlflow.tensorflow.autolog()
        history = self.model.fit(self.dataset[0], self.dataset[1],
                                    validation_data=(self.dataset[2], self.dataset[3]),
                                    epochs=self.hyperparameters["epochs"],
                                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)])
        # TODO log artifacts
        # mlflow.log_artifacts("graphs", self._log_graphs(history))
        return history

    def set_experiment(self):
        mlflow.set_experiment(self.experiment_name)

    def log_params(self):
        mlflow.log_param("model_name", self.model_name)
        for key, value in self.hyperparameters.items():
            mlflow.log_param(key, value)

    
    def log_metrics(self, history):
        for metric_name, metric_values in history.history.items():
            for epoch, value in enumerate(metric_values):
                mlflow.log_metric(f"{metric_name}", value, step=epoch)
    
    # def _log_graphs(self, history):
        # TODO make a function to log graphs
    
        # Save training and validation loss and accuracy plots as artifact
        # import matplotlib.pyplot as plt
        # fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8, 8), dpi=100, sharex=True)
        # ax[0].plot(history.history["loss"], label="train_loss")
        # ax[0].plot(history.history["val_loss"], label="val_loss")
        # ax[0].set_title("Training/Validation Loss")
        # ax[0].set_xlabel("Epoch")
        # ax[0].set_ylabel("Loss")
        # # ax[0].set_grid(True)
        # ax[0].legend()
        
        # ax[1].plot(history.history["accuracy"], label="train_acc")
        # ax[1].plot(history.history["val_accuracy"], label="val_acc")
        # ax[1].set_title("Training/Validation Accuracy")
        # ax[1].set_xlabel("Epoch")
        # ax[1].set_ylabel("Accuracy")
        # # ax[1].set_grid(True)
        # ax[1].legend()
        
        # fig.tight_layout()
        # fig.savefig("graphs.png")
        # plt.close(fig)
        # return "graphs.png"
        
    def log_artifacts(self):
        
        pass

        


In [ ]:
# TODO: Analyze if it is necessary to use gc.collect()
# import gc
# gc.collect()
def registrar_experimento(experimento: Experimento):
    with mlflow.start_run(nested=True):
        experimento.set_experiment()
        experimento.log_params()
        experimento.log_metrics(experimento.fit())
        # experimento._log_graphs(experimento.fit())
        experimento.log_artifacts()

# registrar_experimento(experimento_1)

In [87]:
experimento_1 = Experimento(
    experiment_name = "CNN_autolog",
    model_name = "CNN_basic_1_layer",
    model = create_model_cnn_basic_1_layer(),
    dataset = (X_train, y_train, X_val, y_val),
    hyperparameters = {
        "epochs": 11,
        "filters": 256,
        "kernel_size": 3,
        "pool_size": 2,
        "dropout": 0.1,
        "dense_size": 64,
        "loss": "binary_crossentropy",
        "optimizer": "adam",
        
    },metrics = ["accuracy"],
)

experimento_2 = Experimento(
    experiment_name = "CNN_autolog",
    model_name = "CNN_basic_1_layer",
    model = create_model_cnn_basic_1_layer(),
    dataset = (X_train, y_train, X_val, y_val),
    hyperparameters = {
        "epochs": 17,
        "filters": 256,
        "kernel_size": 3,
        "pool_size": 2,
        "dropout": 0.1,
        "dense_size": 32,
        "loss": "binary_crossentropy",
        "optimizer": "adam",
        
    },metrics = ["accuracy"],
)

In [89]:
%%capture
experimentos = [experimento_1, experimento_2]

for experimento in experimentos:
    registrar_experimento(experimento)
    

INFO:tensorflow:Assets written to: C:\Users\crist\AppData\Local\Temp\tmpvujkpdge\model\data\model\assets
INFO:tensorflow:Assets written to: C:\Users\crist\AppData\Local\Temp\tmp_on0sdbz\model\data\model\assets
